In [126]:
import pandas as pd
import matplotlib as plt
import seaborn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [127]:
# carregar dados de treino e teste
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
# verificar tipos das colunas
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [128]:
# retirar dados irrelevantes
train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [129]:
# one hot encoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [130]:
# verificar nulls no df de treino
train.isnull().sum().sort_values(ascending=False)

Age            177
Embarked_S       0
Embarked_Q       0
Embarked_C       0
Sex_male         0
Sex_female       0
Fare             0
Parch            0
SibSp            0
Pclass           0
Survived         0
PassengerId      0
dtype: int64

In [131]:
# verificar nulls no df de teste
test.isnull().sum().sort_values(ascending=False)

Age            86
Fare            1
Embarked_S      0
Embarked_Q      0
Embarked_C      0
Sex_male        0
Sex_female      0
Parch           0
SibSp           0
Pclass          0
PassengerId     0
dtype: int64

In [132]:
# replace null pela media de idade
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [215]:
# como nao tenho o target do conjunto de teste a ser submetido, 
# tenho que separar o arquivo de treino em treino_treino e treino_teste,
# para medir o score sem vies/vicio
train_train, train_test = train_test_split(train, test_size=0.3, random_state=42)

# separacao das features e target
train_X = train_train.drop('Survived', axis=1)
train_y = train_train['Survived']
test_X = train_test.drop('Survived', axis=1)
test_y = train_test['Survived']

# verificar o score do conjunto de treino
tree = DecisionTreeClassifier(max_depth=4, random_state=2)
tree.fit(train_X, train_y)
tree.score(test_X, test_y)

0.82089552238805974

In [216]:
tree.score(test_X, test_y)

0.82089552238805974

In [135]:
results = pd.DataFrame()
results['PassengerId'] = test['PassengerId']
results['Survived'] = tree.predict(test)

In [136]:
results.to_csv('./output/results.csv', index=False)